In [1]:
import numpy as np
from scipy import integrate
import math

In [2]:
def F(y,x,Sn,Sl):
    beta = (Sn/Sl)**1.5
    f = 1   #2**0.5
    num = (x**2)*(y**2)*(np.exp(-x))*(np.exp(-y))  
    deno = (np.exp(-x)/(beta*f)) + (np.exp(-y)/f)
    return num/deno

In [3]:
def calc_coeff(wan_frag_1,wan_frag_2,wan_frag_1_dist,wan_frag_2_dist):
    conv = 1#1.88973
    conv_1 = 1.88973  #ang to bohr
    inte = []
    C6 = []
    C8 = []
    
    for ele1,ele1_dist in zip(wan_frag_1,wan_frag_1_dist):  #extract spread and distance for elements from frag 1
        count = 0
        for ele2,ele2_dist in zip(wan_frag_2,wan_frag_2_dist): #extract spread and distance for elements from frag 2
            temp_6 = 0
            temp_8 = 0
            if (ele1 != ele2).all():  #remove self interactions
                for wan1,dist1 in zip(ele1,ele1_dist):    #iterate over wannier centers of both fragments
                    for wan2,dist2 in zip(ele2,ele2_dist): 
                        sp_1 = (wan1)**0.5                #sqaure root of spread (in bohr)
                        sp_2 = (wan2)**0.5
                        #print(sp_1,sp_2)
                        rc_1 = sp_1*conv*(3**0.5)*(0.769+(0.5*np.log(sp_1*conv)))
                        rc_2 = sp_2*conv*(3**0.5)*(0.769+(0.5*np.log(sp_2*conv)))
                        lim_x = ((3**0.5)*rc_1)/(sp_1*conv)  #limits of integration
                        lim_y = ((3**0.5)*rc_2)/(sp_2*conv)
        
                        res = integrate.dblquad(F, 0, lim_x, 0, lim_y, args = (sp_1,sp_2))
                        prefac = ((sp_1**1.5)*(sp_2**3)*(conv**4.5))/(2*(3**1.25))
                        temp_6 = temp_6 + (prefac*res[0])     #calculate C6 for current wannier pair
                        temp_8 = temp_8 + ((prefac*res[0])*5*(conv_1**2)*((dist1**2)+(dist2**2)))  #calculate C8 from C6
                C6.append(temp_6)
                C8.append(temp_8)
            else:
                continue
            
    return C6,C8    

In [4]:
def calc_dist(ion,wan):
    #print(ion,wan)
    d = []
    x,y,z = list(map(lambda x: float(x),ion.split()[1:]))
    for i in range(len(wan)):
        wan_x, wan_y, wan_z = list(map(lambda x: float(x),wan[i].split()[1:]))
        dist = ((wan_x-x)**2 + (wan_y-y)**2 + (wan_z-z)**2)**0.5  #calculate distance from ion to wannier
        d.append(dist)
    return d

In [19]:
#driver code for calculating C6 and C8 for unlike fragments (eg. Na-Cl,Cl-O...)

frag = 2                #enter number of fragments
n_frag_1 = 1            #number of atoms in fragment 1
n_frag_2 = 84           #number of atoms in fragment 2
n_wan_at = 4            #number of wannier centers associated with each atom


filename_wan_sort = '/Users/rajorshi/Desktop/test/wannier_sort.xyz' #wannier centers sorted according to ions
filename_sp_sort = '/Users/rajorshi/Desktop/test/spread_sort.xyz' #wannier spreads sorted accoridng to ions
filename_uw = '/Users/rajorshi/Desktop/test/wannier_sort_uw.xyz' #unwrapped coordinates

with open(filename_wan_sort) as f:
    lines = f.readlines()
with open(filename_sp_sort) as f1:
    lines_sp = f1.readlines()
with open(filename_uw) as f2:
    lines_uw = f2.readlines()

n_wan = int(lines_sp[0].split()[0])       #total number of wannier centers
n_atom = int(lines[0].split()[0])-n_wan   #total number of atoms
start_sp = 2
temp_C6 = []
count_fr = 0
C6_final = []
C8_final = []

for i in range(len(lines_uw)):
    s = lines_uw[i].split()
    if len(s)==1:
        count_fr = count_fr + 1
        wan_frag_1 = np.zeros((n_frag_1,n_wan_at)) #2D array with rows as ions and columns are spreads(in bohr**2)
        wan_frag_2 = np.zeros((n_frag_2,n_wan_at))
        wan_frag_1_dist = np.zeros((n_frag_1,n_wan_at)) #2D array with rows as ions and columns are distance of wannier from ion(in ang)
        wan_frag_2_dist = np.zeros((n_frag_2,n_wan_at))
        temp_sp = lines_sp[start_sp:start_sp+n_wan]  #extract spreads for current frame
        start_sp = start_sp+n_wan+2
        coord = lines_uw[i+2:i+2+n_atom]   #extract unwrapped ion coordinates for current frame
        count_1 = 0
        count_2 = 0
        start = 0
        start_wan = i+2+n_atom
        print('Calculating {0} frame'.format(count_fr))
        for j in range(len(coord)):
            s1 = coord[j].split()
            
            if s1[0] == 'Cl':  #element in frag 1
                wan_1,wan_2,wan_3,wan_4 = list(map(lambda x: float(x.split()[0]),temp_sp[start:start+4])) #extract wannier centers
                start = start + 4
                dist = calc_dist(coord[j],lines_uw[start_wan:start_wan+4]) #calculate distance of wannier from ion
                start_wan = start_wan + 4
                
                wan_frag_1[count_1][:] = wan_1,wan_2,wan_3,wan_4  
                wan_frag_1_dist[count_1][:] = dist[0],dist[1],dist[2],dist[3]
                count_1 = count_1 + 1
            elif s1[0] == 'O':  #element in frag 2
                
                wan_1,wan_2,wan_3,wan_4 = list(map(lambda x: float(x.split()[0]),temp_sp[start:start+4]))
                start = start + 4
                dist = calc_dist(coord[j],lines_uw[start_wan:start_wan+4])
                start_wan = start_wan + 4
                
                wan_frag_2[count_2][:] = wan_1,wan_2,wan_3,wan_4
                wan_frag_2_dist[count_2][:] = dist[0],dist[1],dist[2],dist[3]
                count_2 = count_2 + 1

        temp_C6,temp_C8=calc_coeff(wan_frag_1,wan_frag_2,wan_frag_1_dist,wan_frag_2_dist) #cal C6 and C8 for current frame
        C6_final.extend(temp_C6)
        C8_final.extend(temp_C8)
        

Calculating 1 frame
Calculating 2 frame
Calculating 3 frame
Calculating 4 frame
Calculating 5 frame
Calculating 6 frame
Calculating 7 frame
Calculating 8 frame
Calculating 9 frame
Calculating 10 frame
Calculating 11 frame
Calculating 12 frame
Calculating 13 frame
Calculating 14 frame
Calculating 15 frame
Calculating 16 frame
Calculating 17 frame
Calculating 18 frame
Calculating 19 frame
Calculating 20 frame
Calculating 21 frame
Calculating 22 frame
Calculating 23 frame
Calculating 24 frame
Calculating 25 frame
Calculating 26 frame
Calculating 27 frame
Calculating 28 frame
Calculating 29 frame
Calculating 30 frame
Calculating 31 frame
Calculating 32 frame
Calculating 33 frame
Calculating 34 frame
Calculating 35 frame
Calculating 36 frame
Calculating 37 frame
Calculating 38 frame
Calculating 39 frame
Calculating 40 frame
Calculating 41 frame
Calculating 42 frame
Calculating 43 frame
Calculating 44 frame
Calculating 45 frame
Calculating 46 frame
Calculating 47 frame
Calculating 48 frame
C

In [20]:
((2**0.5)*(sum(C6_final)/len(C6_final)))*(0.529117**6)

2.22821506474285

In [21]:
((2**0.5)*(sum(C8_final)/len(C8_final)))*(0.529117**8)

4.3471990300996275